# 10章　文の意味の解析

ここまで、コンピュータの力が大規模にテキストを処理するのにどれくらい役に立つかということについて見てきた。そして我々はすでに構文解析器と素性ベースの文法の機構を使えるようになっている。そこで今度は、文の意味を解析することによって、何か同じくらい意義があることができないだろうか。

## 本書の目的
１．どのようにして自然言語の意味を表現し、コンピュータが処理できるようにするか。

２．制約のない文に対してどのように意味表現を関連付けるか。

３．文の意味表現を知識源に結びつけるプログラムをどのようにして利用するか。

これらの質問に答える過程で、論理的意味論の分野における形式的な手法を学ぶことができる。さらにそうした手法を用いることで、様々な情報を格納しているデータベースに問い合わせを行う方法についても学ぶことになるだろう。

## 10.1 自然言語理解

## 10.1.1 データベースに対する問合せ

#### システムに問い合わせるための入力として英語を使う
9章で説明した素性ベースの文法形式を用いると，英語からSQLへの翻訳が容易になる．
#### 文法spl0.fcfg
文を構文解析するのと同時に，文の意味表現をどのように組み立てるかを例示している．各句構造規則には，素性SEMに対する値を組み立てるための方法が付随している．いずれの場合も，文字列連結演算を用いて，この構成素に対応する値をつなぎ合わせることで親の構成素の値を作っている．

In [11]:
# -*- coding:utf-8 -*-
import nltk 
# book_grammars download
nltk.data.show_cfg('sql0.fcfg')

% start S
S[SEM=(?np + WHERE + ?vp)] -> NP[SEM=?np] VP[SEM=?vp]
VP[SEM=(?v + ?pp)] -> IV[SEM=?v] PP[SEM=?pp]
VP[SEM=(?v + ?ap)] -> IV[SEM=?v] AP[SEM=?ap]
NP[SEM=(?det + ?n)] -> Det[SEM=?det] N[SEM=?n]
PP[SEM=(?p + ?np)] -> P[SEM=?p] NP[SEM=?np]
AP[SEM=?pp] -> A[SEM=?a] PP[SEM=?pp]
NP[SEM='Country="greece"'] -> 'Greece'
NP[SEM='Country="china"'] -> 'China'
Det[SEM='SELECT'] -> 'Which' | 'What'
N[SEM='City FROM city_table'] -> 'cities'
IV[SEM=''] -> 'are'
A[SEM=''] -> 'located'
P[SEM=''] -> 'in'


In [15]:
# クエリを構文解析してSQLに変換
from nltk import load_parser
cp = load_parser('sql0.fcfg')
query = 'What cities are located in China'
trees = list(cp.parse(query.split()))
answer = trees[0].label()['SEM']
answer = [s for s in answer if s]
print(answer)
q = ' '.join(answer)
print(q)

['SELECT', 'City FROM city_table', 'WHERE', 'Country="china"']
SELECT City FROM city_table WHERE Country="china"


In [20]:
# city.dbに対してクエリを実行し，結果を得る．
from nltk.sem import chat80
rows = chat80.sql_query('corpora/city_database/city.db', q)
for r in rows:
    print(r[0], end=" ")

canton chungking dairen harbin kowloon mukden peking shanghai sian tientsin 

ここではコンピュータが自然言語で書かれた問合せに対して有用なデータを返すというタスクを定義して，英語の小さなサブセットをSQLに翻訳することによってこれを実装した．

PythonはSQLクエリをデータベースに対して実行できるため，NLTKのコードはSQLをすでに「理解」できている．つまり「What cities are located in Chiha」というようなクエリを「理解」することもできる，

### 10.1.2 自然言語，意味論，論理

#### 意味論における2つの概念
- 表明文が「ある状況において真もしくは偽」
- 限定された名詞句や固有名詞は「世界における実態に対応する」

状況における真偽値という概念は，推論のための強力なツールになる．とりわけ文のセットを考えると，それらがある状況において同時に真になるかどうかを問うことができる．文によって整合である2文と不整合な2文というのが実世界には存在するが，推論能力において言及すれば，実世界において何が真で何が偽であるかに依存しない．つまり推論能力において正しく真偽を判別するためには実世界において何が真で何が偽であるかという事実についても知る必要がある．

論理言語の構文は整合か不整合かを判別する特徴が形式的に明確になるよう設計されている．そのため整合性のような性質を決定することはコンピュータによって実行可能なタスクとして還元できる．

## 10.2 命題論理学

In [24]:
# nltkにおいて命題論理を扱う
import nltk
nltk.boolean_ops

<function nltk.sem.logic.boolean_ops()>

In [ ]:
read_expr = nltk.sem.Expression.fromstring
read_expr('-(P & Q)')
read_expr('(P & Q)')
read_expr('P | (R -> Q)')
read_expr('P <-> --P')

In [40]:
val = nltk.Valuation([('P', True), ('Q', True), ('R', False)])
val['P']
val['R']
val['Q']

True

In [42]:
# 現時点ではdomとgパラメータは無視しておく．
dom = set([])
g = nltk.Assignment(dom)

In [44]:
m = nltk.Model(dom, val)
print(m.evaluate('(P & Q)', g))
print(m.evaluate('-(P & Q)', g))
print(m.evaluate('(P & R)', g))
print(m.evaluate('(P | R)', g))

True
False
False
True


ここまでは英語文を命題論理に翻訳してきた．しかし原子的な文をPやQなどの文字で表現することしかできてないので，内部構造まで掘り下げられていない．これは原子文を主語や目的語，述語に分割することに意味的な利点がないとしているようなものである．

#### さらに複雑な論理へ
一回述語論理と呼ばれる論理へと移行する．

## 10.3 一階述語論理

#### 計算意味論的利点
一階述語理論は意味論の多くの側面を表現できるうえに，この論理用の自動推論を実行する素晴らしいシステムはすぐに手に入る．

In [48]:
read_expr = nltk.sem.Expression.fromstring
expr = read_expr('walk(angus)', type_check=True)
expr.argument
expr.argument.type
expr.function
expr.function.type

<e,?>

In [49]:
# 型チェッカーはなるべく多くの方を推定しようとするためシグネチャを作成
sig = {'walk': '<e, t>'}
expr = read_expr('walk(angus)', signature=sig)
expr.function.type

e

### 10.3.1 構文

### 10.3.2 一階定理証明

### 10.3.3 一階述語論理言語のまとめ

### 10.3.4 モデルにおける真偽値

### 10.3.5 個体変数と代入

### 10.3.6 量化子

### 10.3.7 量化子スコープの曖昧さ

### 10.3.8 モデルの構築

## 10.4 英語文の意味論

## 10.5 談話意味論

## 10.6 まとめ